# Estimator primitive

Understand the input and output for the Estimator primitive.

## Overview


The Estimator primitive returns an observable expectation value generated by given circuits executed on the target backend.

The maximum execution time is 18000 seconds (5 hours).

## Parameters


- **Input parameters**:
    - **run_options**:
        - **Type**: object
        - **Description**: A collection of key-value pairs identifying the execution options, such as shots.
        - **Required**: False
    - **circuit_indices**:
        - **Type**: array
        - **Description**: Indexes of the circuits to evaluate.
        - **Required**: True
    - **parameter_values**
        - **Type**: array
        - **Description**: Concrete parameters to be bound.
        - **Required**: False
    - **skip_transpilation**:
        - **Description**: Skip circuit transpilation. The default value is False.
        - **Required**: False.   
    - **circuits**:
        - **Description**: A QuantumCircuit or list of QuantumCircuits. The circuits can be parameterized.
        - **Required**: True
    - **parameters**
        - **Type**: array
        - **Description**: Parameters of the quantum circuits.
        - **Required**: False  
    - **observables:**
        - **Type**: ['object', 'array']
        - **Description**: The Hamiltonians to be evaluated.
        - **Required**: False
    - **observable_indices**:
        - **Type**: array
        - **Description**: A list of observable indices. It must have the same length as circuit_indices and parameter_values.
        - **Required**: True   
    - **options**        
        - **Description**: Additional parameters, such as the backend to use.
        - **Required**: False   
- **Returns**:
   - **metadata**:
        - **Type**: array
        - **Description**: Additional metadata.  
        - **Required**: False
   - **value**:
       - **Description**: Estimated expectation values. This is a numpy array. The i<sup>th</sup> element is calculated using the circuit and observable indexed by the i<sup>th</sup> circuit_indices and i<sup>th</sup> observable_indices, and bound with i<sup>th</sup> parameter_values.
       - **Type**: array
       - **Required**: False

## Example

In [5]:
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import SparsePauliOp

service = QiskitRuntimeService()

psi1 = RealAmplitudes(num_qubits=2, reps=2)
psi2 = RealAmplitudes(num_qubits=2, reps=3)

H1 = SparsePauliOp.from_list([("II", 1), ("IZ", 2), ("XI", 3)])
H2 = SparsePauliOp.from_list([("IZ", 1)])
H3 = SparsePauliOp.from_list([("ZI", 1), ("ZZ", 1)])

with Estimator(
        circuits=[psi1, psi2],
        observables=[H1, H2, H3],
        service=service, 
        options=< "backend": "ibmq_qasm_simulator" },
        ) as estimator:
    theta1 = [0, 1, 1, 2, 3, 5]
    theta2 = [0, 1, 1, 2, 3, 5, 8, 13]
    theta3 = [1, 2, 3, 4, 5, 6]

    # calculate [ <psi1(theta1)|H1|psi1(theta1)> ]
    psi1_H1_result = estimator(circuit_indices=[0], observable_indices=[0], parameter_values=[theta1])
    print(psi1_H1_result)

    # calculate [ <psi1(theta1)|H2|psi1(theta1)>, <psi1(theta1)|H3|psi1(theta1)> ]
    psi1_H23_result = estimator(circuit_indices=[0, 0], observable_indices=[1, 2], parameter_values=[theta1]*2)
    print(psi1_H23_result)

    # calculate [ <psi2(theta2)|H2|psi2(theta2)> ]
    # Note that you don't need to specify the labels "circuit_indices", "observable_indices", or "parameter_values", as long as they are specified in that order.
    psi2_H2_result = estimator([1], [1], [theta2])
    print(psi2_H2_result)

    # calculate [ <psi1(theta1)|H1|psi1(theta1)>, <psi1(theta3)|H1|psi1(theta3)> ]
    psi1_H1_result2 = estimator([0, 0], [0, 0], [theta1, theta3])
    print(psi1_H1_result2)

    # calculate [ <psi1(theta1)|H1|psi1(theta1)>,
    #             <psi2(theta2)|H2|psi2(theta2)>,
    #             <psi1(theta3)|H3|psi1(theta3)> ]
    psi12_H23_result = estimator([0, 1, 0], [0, 1, 2], [theta1, theta2, theta3])
    print(psi12_H23_result)

EstimatorResult(values=array([1.59570312]), metadata=[{'variance': 8.525310516357422, 'shots': 1024}])
EstimatorResult(values=array([-0.55859375,  0.05273438]), metadata=[{'variance': 0.6879730224609375, 'shots': 1024}, {'variance': 1.9970054626464844, 'shots': 1024}])
EstimatorResult(values=array([0.13671875]), metadata=[{'variance': 0.9813079833984375, 'shots': 1024}])
EstimatorResult(values=array([1.55078125, 1.13671875]), metadata=[{'variance': 8.991500854492188, 'shots': 1024}, {'variance': 11.974227905273438, 'shots': 1024}])
EstimatorResult(values=array([ 1.60546875,  0.12304688, -1.0234375 ]), metadata=[{'variance': 8.304977416992188, 'shots': 1024}, {'variance': 0.9848594665527344, 'shots': 1024}, {'variance': 1.2834091186523438, 'shots': 1024}])
